## Linear Systems and Iterative Solutions
---
**Objectives and Plan**

1. Linear Systems of Equations and Gaussian Elimination with pivoting
1. LU Decomposition of A
1. Iterative solution of Linear Systems

In [41]:

#IMPORT
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#%matplotlib inline

#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

## Set a seed for the random number generator
np.random.seed(100)

## Linear System of Equations
---
Consider the following system of $m$ linear equations in $n$ variables.
\begin{align}
a_{11} x_1 + a_{12} x_2  + \cdots + a_{1n} x_n  &= b_1 \\
a_{21} x_1 + a_{22} x_2  + \cdots + a_{2n} x_n  &= b_2 \\
 \vdots \qquad \qquad   & \ \\
a_{m1} x_1 + a_{m2} x_2  + \cdots + a_{mn} x_n  &= b_ m,
\end{align}

-  The solution of a linear system represents the **point of intersection of hyperplanes** given by the lienar equations.


-  The solution also represents the **linear coding of the columns** of a matrix $A$ to get a vector $b$ in the column space ($\mathcal{C}(A)$).
 
$$
 x_1 \begin{pmatrix}a_{11}\\a_{21}\\ \vdots \\a_{m1}\end{pmatrix} +
 x_2 \begin{pmatrix}a_{12}\\a_{22}\\ \vdots \\a_{m2}\end{pmatrix} +
 \cdots +
 x_n \begin{pmatrix}a_{1n}\\a_{2n}\\ \vdots \\a_{mn}\end{pmatrix}
 =
 \begin{pmatrix}b_1\\b_2\\ \vdots \\b_m\end{pmatrix}
 $$
 
 or simply as
 $$
 x_1 A_{:1}+x_1 A_{:2}+ \cdots +x_1 A_{:n} = \mathbf{b}
 $$
 

- The system could be represented in a compact form as $Ax = b$, where 
 $$
 A=
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix},\quad
\mathbf{x}=
\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix},\quad
\mathbf{b}=
\begin{pmatrix}
b_1 \\
b_2 \\
\vdots \\
b_m
\end{pmatrix}
$$

- The system is called...
  - **overdetermined** when $m > n$.
  - **underdetermined** when $ m < n$.

## Gaussian elimination with scaled-row partial pivoting
---
  - What is the need for pivoting?
  - How can you find a factorization of the matrix by using the following code.
  - How can you modify the code to find the determinant of a matrix?
  - How can you modify the following code to solve a linear system?
  - How can you modify the code to find the inverse of a matrix?


In [42]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_srpp(X, verbose=False):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    A = np.copy(X)
    m,n = A.shape
    swap=0;
    
    # The initial ordering of rows
    p = list(range(m))
    if verbose:
        print("permutation vector initialized to: ",p)
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1) 
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(m-1):              
        if verbose:
            print("Scaling Vector: ",s)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange elements in the permutation vector if needed
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            swap+=1;
                        
        if verbose:
            print("\nPivot Element: {0:.4f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular or there is a zero pivot.")        
        # Check the new order of rows
        if verbose:
            print("permutation vector: ",p)
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, m):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]            
            #Save the multiplier z in A itself. You can save this in L also
            A[p[i],k] = z          
            #Elimination operation: Changes all elements in a row simultaneously
            A[p[i],k+1:] = A[p[i],k+1:] - z*A[p[k],k+1:]
            
        if verbose:
            print("\n After PASS {}=========: \n".format(k+1), A)
    return A, p, swap

In [43]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
A0 = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
#A = np.array([[5, 4, 7, 6, 9], [7, 8, 9, 9, 8], [2, 3, 5, 9, 8], [3, 1, 7, 5, 6], [9, 1, 3, 7, 3]], dtype=float)
#A0=np.array([[1, 0, 2, 1],[4, -9, 2, 1],[8, 16, 6, 5],[2, 3, 2,1]], dtype=float)
print("\n Given A: \n ",A0)


 Given A: 
  [[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]


In [31]:
A,p,swap =GE_srpp(A0) #You need to pass a copy, otherwise the original A0 is chaneged.
m=A.shape[0]

In [44]:
L = np.tril(A[p,:], -1)+np.eye(n)
U = np.triu(A[p,:]) # YOu can solve Ux = b[p] by back substitution
P = np.eye(n)[p,:]
print("\n After Gaussian Elimination with SRPP: \n", A)


 After Gaussian Elimination with SRPP: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -1.23076923 -0.53846154]
 [ 3.         -2.          1.        ]]


In [47]:
print("\n The permutation Vector is: \n", p)
print("\n Permutation matrix, P:\n ", P)
print("\n Upper triangular, U:\n ", U)
print("\n Lower triangular, L:\n", L)
print("Sanity Check: norm of PA-LU",np.linalg.norm(P@A0-L@U))


 The permutation Vector is: 
 [2, 0, 1]

 Permutation matrix, P:
  [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

 Upper triangular, U:
  [[ 3.         -2.          1.        ]
 [ 0.          4.33333333 -6.66666667]
 [ 0.          0.         -0.53846154]]

 Lower triangular, L:
 [[ 1.          0.          0.        ]
 [ 0.66666667  1.          0.        ]
 [ 0.33333333 -1.23076923  1.        ]]
Sanity Check: norm of PA-LU 0.0


In [36]:
A0
print(A0)
b=np.array([-1,3,2], dtype=float).reshape(3,-1)
augA = np.hstack((A0,b))
print(augA)
print(np.hstack((A0,np.eye(3))))

[[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]
[[ 2.  3. -6. -1.]
 [ 1. -6.  8.  3.]
 [ 3. -2.  1.  2.]]
[[ 2.  3. -6.  1.  0.  0.]
 [ 1. -6.  8.  0.  1.  0.]
 [ 3. -2.  1.  0.  0.  1.]]


In [37]:
newA,p,swap = GE_srpp(augA)
newU = np.triu(newA[p,:3])
newb = newA[p,3]
print(back_sub(newU, newb))

[1. 1. 1.]


In [35]:
print("The Permutation Matrix, P:\n",P)
print("Error Norm \n", np.linalg.norm(np.dot(L,U) - np.dot(P,A0)))
print(np.dot(L,U))
print(np.dot(P,A0))

The Permutation Matrix, P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
Error Norm 
 0.0
[[ 3. -2.  1.]
 [ 2.  3. -6.]
 [ 1. -6.  8.]]
[[ 3. -2.  1.]
 [ 2.  3. -6.]
 [ 1. -6.  8.]]


### LU Factorization 
---
One can use the Gaussian elimination with partial pivoting to decompose the matrix A as follows
$$
P A = L U,\text{ or }\ A = P^T L U;
$$
where $P$ is a permutation matrix, $L$ is a unit lower-triangular matrix and $U$ is an upper triangular matrix.

In [19]:
print("\n Upper triangular, U:\n ", np.triu(A[p,:]))
print("\n Lower triangular, L:\n", np.tril(A[p,:], -1)+np.eye(3))
print("The Permutation Matrix, P:\n",np.eye(3)[p,:])


 Upper triangular, U:
  [[ 3.         -2.          1.        ]
 [ 0.          4.33333333 -6.66666667]
 [ 0.          0.         -0.53846154]]

 Lower triangular, L:
 [[ 1.          0.          0.        ]
 [ 0.66666667  1.          0.        ]
 [ 0.33333333 -1.23076923  1.        ]]
The Permutation Matrix, P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


### Back-substitution
---
One can solve a square linear system with upper triangular structure (REF) $$U{\bf x} = {\bf b} $$ by using back-substitution, where 
$$
U = \begin{bmatrix}
  u_{1,1} & u_{1,2} & u_{1,3} & \ldots &   u_{1,n} \\
        0  & u_{2,2} & u_{2,3} & \ldots &   u_{2,n} \\
        \vdots  &  \vdots       &  \ddots & \ddots &    \vdots \\
        0  &    0     &     \cdots    & u_{n-1,n-1} & u_{n-1,n} \\
        0 &     0    &   \cdots      &   \cdots     &   u_{n,n}
\end{bmatrix}
$$
The backward substitution algorithm finds the unknowns starting from the last $x_n$ to the first $x_1$.
$$
\begin{align}
  x_n &= \frac{b_n}{u_{n,n}}, \\
  x_{n-1} &= \frac{b_2 - u_{n-1,n} x_n}{u_{n-1,n-1}}, \\
      &\ \ \vdots \\
  x_k &= \frac{b_k - \sum\limits_{i=n, step = -1}^{k+1} u_{k,i}x_i}{u_{k,k}}.
\end{align}
$$

In [17]:
# In the following function A is supposed to be upper triangular systems UX=b
def back_sub(A, b):
    n = A.shape[0]
    #Solution will be saved in variable x
    x = np.zeros_like(b, dtype=float)

    # Back-substitution
    ##last variable is found first
    x[n-1] = b[n-1] / A[n-1,n-1]
    ## Find the remaining n-1 variables from last to first
    for k in range(n-2,-1,-1):
        known_sums = np.dot(A[k,k+1:],x[k+1:])
        x[k] = (b[k] - known_sums) / A[k,k]
    return x

**Example** Solve the following upper triangular system
$$
\begin{pmatrix}
1 & 2 & 3 & 1\\
0 & 4 & 5 &3\\
0 & 0 & 6 & 2\\
0 & 0 & 0& 5
\end{pmatrix}
\begin{pmatrix}
x_1\\x_2\\x_3\\x_4
\end{pmatrix} = \begin{pmatrix}
7\\12\\8\\5
\end{pmatrix}
$$
By using backsubstitution 

$x_4 = b_1/u_{11} =5/5 1$, 

$x_3 = (8 - 2x_4)/u_{22}= (8 - 2(1))/6=1$, 

$x_2=(12-5x_3-3x_4)/u_{33}=(12-5(1)-3(1))/4=1$, 

$x_1 = (7 - 2x_2-3x_3-1x_4)/u_{11}= (7 - 2(1)-3(1)-1(1))/1=1$.

In [14]:
# Here is an example for backward substitution
A =np.array([[1,2,3,1],
           [0,4,5,3],
           [0,0,6,2],
            [0,0,0,5]],dtype=float)
b=np.array([7,12,8,5],dtype=float)
print(back_sub(A,b))

[1. 1. 1. 1.]


## Iterative Methods for Linear Systems
---
Refer to John Foster's page on [iterative methods](https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_IterativeSolvers.html).


### Jacobi Method



>  Initialize the iterative solution vector $x^{(0)}$ randomly, or with the zero vector,

>  for k=0:maxIteration, update every element until convergece

>> for i=1:n
$$
x^{(k+1)}_i  = \frac{1}{a_{ii}} \left(b_i -\sum_{j\ne i}a_{ij}x^{(k)}_j\right).
$$

In [48]:
# You can modify this code to answer the following
'''
Jacobi's iteration method for solving the system of equations Ax=b.
p0 is the initialization for the iteration.
'''
def jacobi(A, b, p0, tol, maxIter=100):
    n=len(A)
    p = p0

    for k in range(maxIter):
        p_old = p.copy() # In python assignment is not the same as copy
        
        # Update every component of iterant p
        for i in range(n):
            sumi = b[i];
            for j in range(n):
                if i==j: # Diagonal elements are not included in Jacobi
                    continue;
                sumi = sumi - A[i,j] * p_old[j]
            p[i] = sumi/A[i,i]
                
        my_error = np.linalg.norm(p-p_old)/n
       #  print("Relative error in iteration", k+1,":",rel_error)
        if my_error<tol:
            print("TOLERANCE MET BEFORE MAX-ITERATION.")
            print("Total number of iterations:",k)
            break
    return p;

In [49]:
# Example System - This example is from the WikiPedia page on gauss Seidel
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]],dtype=float)
b = np.array([6, 25, -11, 15],dtype=float)

In [51]:
x_0 = np.array([0, 0, 0, 0],dtype=float)
soln  = jacobi(A,b,x_0,0.000000001,100)
print("The solution is:\n",soln)

TOLERANCE MET BEFORE MAX-ITERATION.
Total number of iterations: 24
The solution is:
 [ 1.  2. -1.  1.]


In [63]:
n=1000
A2 = 3*n * np.eye(n) + 1.0* np.random.randint(0,5,size=(n,n));

In [64]:
known_x = 1.0*np.random.randint(1,5,(n,1));
b2 = np.dot(A2,known_x);

In [65]:
soln  = jacobi(A2,b2,1.0*np.random.randint(1,2,(n,1)),0.0000001,100)
#print("The solution is:\n",soln)
print("The error:\n",np.linalg.norm(known_x-soln))

TOLERANCE MET BEFORE MAX-ITERATION.
Total number of iterations: 34
The error:
 3.2532451921552304e-05


In [5]:
A5=np.array([[5,4,7,6,9],
           [7,8,9,9,8],
           [2,3,5,9,8],
           [3,1,7,5,6],
           [9,1,3,7,3]], dtype=float)
A_ = GE_srpp(np.copy(A5))

permutation vector initialized:  [0, 1, 2, 3, 4]
Scaling Vector:  [9. 9. 9. 7. 9.]

 Pivot Element: 9.00 

permutation vector:  [4, 1, 2, 3, 0]

 After PASS 1=========: 
 [[0.55555556 3.44444444 5.33333333 2.11111111 7.33333333]
 [0.77777778 7.22222222 6.66666667 3.55555556 5.66666667]
 [0.22222222 2.77777778 4.33333333 7.44444444 7.33333333]
 [0.33333333 0.66666667 6.         2.66666667 5.        ]
 [9.         1.         3.         7.         3.        ]]
Scaling Vector:  [9. 9. 9. 7. 9.]

 Pivot Element: 7.22 

permutation vector:  [4, 1, 2, 3, 0]

 After PASS 2=========: 
 [[0.55555556 0.47692308 2.15384615 0.41538462 4.63076923]
 [0.77777778 7.22222222 6.66666667 3.55555556 5.66666667]
 [0.22222222 0.38461538 1.76923077 6.07692308 5.15384615]
 [0.33333333 0.09230769 5.38461538 2.33846154 4.47692308]
 [9.         1.         3.         7.         3.        ]]
Scaling Vector:  [9. 9. 9. 7. 9.]

 Pivot Element: 5.38 

permutation vector:  [4, 1, 3, 2, 0]

 After PASS 3=========: 
 [[ 

In [2]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [15]:
!jt -t grade3